In [2]:
import pandas as pd
import requests
import warnings
import time
import multiprocessing
import threading
from concurrent.futures import ThreadPoolExecutor
# from google.colab import files
warnings.filterwarnings('ignore', message='Unverified HTTPS request')

In [3]:
df = pd.read_csv('ipa_validation_batch1.csv')

In [6]:
# char_list_dependent=['̯', 'ʰ', 'ʱ', 'ʰ', '̪', '̪', '̃', '̪', 'ʰ',"ʲ",'ʰ', 'ʱ', 'ʲ', 'ʷ', 'ː', '̃', '̆', '̑', '̪', '̯', '͡',]
char_list_unique_all=['F', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'z', 'ø', 'ŋ', 'ɐ', 'ɔ', 'ɛ', 'ɟ', 'ɡ', 'ɦ', 'ɪ', 'ɽ', 'ɾ', 'ʃ', 'ʰ', 'ʱ', 'ʲ', 'ʷ', 'ː', '̃', '̆', '̑', '̪', '̯', '͡']

char_list_unique=['F','a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'z', 'ø', 'ŋ', 'ɐ', 'ɔ', 'ɛ', 'ɟ', 'ɡ', 'ɦ', 'ɪ', 'ɽ', 'ɾ', 'ʃ', 'ʰ', 'ʱ', 'ʲ', 'ʷ', 'ː', '̃', '̆', '̑', '̪', '̯', '͡',]

ipa_vowels=['a', 'e', 'i', 'o', 'u', 'ɐ', 'ɔ', 'ɛ', 'ɪ']
ipa_dependent=['̃', 'ː', '̯', '̑', 'ʰ', 'ʱ', '̪', 'ʷ', '͡', 'ʲ', '̆']
ipa_short_vowels=['ɪ', 'ɛ', 'æ', 'ɑ', 'ɔ', 'ʊ', 'ʌ', 'ə']
ipa_extra=['õ']

ipa_consonants=['b', 'c', 'd', 'f', 'g', 'h', 'j', 'k', 'l', 'm', 'n', 'p', 'r', 's', 't', 'v', 'z', 'ŋ', 'ɟ', 'ɡ', 'ɦ', 'ɽ', 'ɾ', 'ʃ']
# ipa_consonants=['b', 'c', 'd', 'f', 'g', 'h', 'j', 'k', 'l', 'm', 'n', 'p', 'r', 's', 't', 'v', 'z', 'ø', 'ŋ', 'ɟ', 'ɡ', 'ɦ', 'ɽ', 'ɾ', 'ʃ', 'ʰ', 'ʱ', 'ʲ', 'ʷ']


In [7]:
sub_2000_3000 = df.iloc[2000:3000]
sub_2000_3000["ɪpas"]
for row in df.itertuples():
  if type(row.ɪpas)==float:
    continue
  for ch in row.ɪpas:
    if ch not in char_list_unique_all:
      char_list_unique_all.append(ch)


  
    # print(f"{row.Index} {row.ɪpas} {row.words}")
    # break
    # if ch =="ø":
    #   print(f"{row.Index} {row.ɪpas} {row.words}")
    #   break

In [9]:
print(char_list_unique)

['F', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'z', 'ø', 'ŋ', 'ɐ', 'ɔ', 'ɛ', 'ɟ', 'ɡ', 'ɦ', 'ɪ', 'ɽ', 'ɾ', 'ʃ', 'ʰ', 'ʱ', 'ʲ', 'ʷ', 'ː', '̃', '̆', '̑', '̪', '̯', '͡']


In [25]:
test="ɔʃɔhɐʲot̪t̪o"
for ch in test:
  print(ch)

ɔ
ʃ
ɔ
h
ɐ
ʲ
o
t
̪
t
̪
o


In [11]:
dependent_ipa="u̯gʰbʱɽʰt̪t̪õd̪ʰ"
dependent_ipa_list=["ʰ","̃","ʱ","","̯","̪","̱","̊","̚","̈","̥","̬","̤","̰","̼","̹","̜","̟","̠","̽","̩","̯","̮","̪","̤","̞","̝","̘","̙","̆","̑","̊","̌",""]
list_of_char=[]
for ch in dependent_ipa:

  list_of_char.append(ch)

print(list_of_char)


['u', '̯', 'g', 'ʰ', 'b', 'ʱ', 'ɽ', 'ʰ', 't', '̪', 't', '̪', 'o', '̃', 'd', '̪', 'ʰ']


In [20]:
list_of_char.remove("d")
print(list_of_char)

['̯', 'ʰ', 'ʱ', 'ʰ', '̪', '̪', '̃', '̪', 'ʰ']
